In [1]:
import json

from kafka import KafkaConsumer

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [2]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Josh',
    last_name='Greenert'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Josh',
 'last_name': 'Greenert',
 'client_id': 'GreenertJosh',
 'topic_prefix': 'GreenertJosh'}

Create a consumer without subscribing to any particular topic

In [3]:
general_consumer = KafkaConsumer(
    bootstrap_servers=config['bootstrap_servers']
)

List all topics you are currently allowed to view

In [4]:
general_consumer.topics()

{'AvinazGabriel-accelerations',
 'AvinazGabriel-locations',
 'ClaytonDaniel-accelerations',
 'ClaytonDaniel-locations',
 'CosentiniVictor-accelerations',
 'CosentiniVictor-locations',
 'DoeJohn-accelerations',
 'DoeJohn-locations',
 'GreenertJosh-accelerations',
 'GreenertJosh-locations',
 'HanPhil-accelerations',
 'HanPhil-locations',
 'MeyerJake-acceleration',
 'MeyerJake-accelerations',
 'MeyerJake-locations',
 'OehlerAngie-accelerations',
 'OehlerAngie-locations',
 'Oehler_Angie-locations',
 'OmprakashSantosh-accelerations',
 'OmprakashSantosh-locations',
 'PatelMithil-accelerations',
 'PatelMithil-locations',
 'ShermanMilan-accelerations',
 'ShermanMilan-locations',
 'TabbalatAbed-accelerations',
 'TabbalatAbed-locations',
 'TabbalatAbed-simple',
 'WeatherfordPatrick-accelerations',
 'WeatherfordPatrick-locations'}

Close the consumer, waiting indefinitely for any needed cleanup.

In [5]:
general_consumer.close()

In [6]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['locations', 'accelerations'])

Gets a list of this consumer's current subscriptions

In [7]:
consumer.subscription()

{'GreenertJosh-accelerations', 'GreenertJosh-locations'}

The following function prints messages from the current consumer subscriptions. It will continue until manually stopped. 

In [ ]:
def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])

                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                print(msg_value)
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")
        
print_messages()

Message metadata:
  Topic: GreenertJosh-locations
  Partition: 0
  Offset: 0
Message Key: e56751e312544e0daabbf3ac3dd4b69a
Message Value:
  {
    "id": "85c61911b7fe2ced1000c33c9e932706",
    "ride_id": "6760ffa3f41908695d1405b776c3e8d5",
    "uuid": "dad7eae44e784b549c8c5a3aa051a8c7",
    "timestamp": "1970-01-01T00:25:07.320Z",
    "offset": 1.0779125295566454,
    "course": 158.203125,
    "latitude": 40.677641336844,
    "longitude": -73.81793000742218,
    "geohash": "dr5x2jpkmtcy",
    "speed": 2.119999885559082,
    "accuracy": 10.0,
    "timelapse": false,
    "filename": "d745b92f-aefd-467d-9121-7a71308e8d6d.mov",
    "t": 0.0
  }

Message metadata:
  Topic: GreenertJosh-locations
  Partition: 0
  Offset: 1
Message Key: c0bf8b8240bc447184a34bc2820b1ab7
Message Value:
  {
    "id": "58682c5d48cad9d9e103431d773615bf",
    "ride_id": "c9a2b46c9aa515b632eddc45c4868482",
    "uuid": "19b9aa10588646b3bf22c9b4865a7995",
    "timestamp": "1970-01-01T00:25:03.882Z",
    "offset": 1.525

Close the consumer, waiting indefinitely for any needed cleanup.

In [ ]:
# Close the consumer since it's done processing the messages.
consumer.close()